O objetivo do código aqui é determinar a quantidade de lags a serem utilizados

In [28]:
import utils
import epftools as epf
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError, MeanSquaredError, MeanAbsoluteError
from sklearn.ensemble import RandomForestRegressor
from sktime.forecasting.compose import make_reduction
import pandas as pd

In [4]:
df = utils.read_and_change_index("dados/lags_transformados.csv")
df.head()

,ipca,ipca_15,selic,m1,m2,m3,m4,ibovespa,cambio,cambio_real,ind_varejo,ind_varejo_ampl,ind_industria,ibc_br,capacidade_industria,salario_minimo,energia_total,energia_residencial,energia_industrial,energia_comercial,energia_outros,oil_usa,oil_eu,pib,bens_capital,bens_int,bens_cons_d,bens_cons_nd,prod_veiculos,prod_caminhoes,prod_onibus,result_primario,el_nino,epu_br,epu_usa,ind_conf_cons,desemprego,rendimento,crb_cmdt,crb_food,...,rendimento_lag1,rendimento_lag2,rendimento_lag3,rendimento_lag4,rendimento_lag5,rendimento_lag6,rendimento_lag7,rendimento_lag8,rendimento_lag9,rendimento_lag10,crb_cmdt_lag1,crb_cmdt_lag2,crb_cmdt_lag3,crb_cmdt_lag4,crb_cmdt_lag5,crb_cmdt_lag6,crb_cmdt_lag7,crb_cmdt_lag8,crb_cmdt_lag9,crb_cmdt_lag10,crb_food_lag1,crb_food_lag2,crb_food_lag3,crb_food_lag4,crb_food_lag5,crb_food_lag6,crb_food_lag7,crb_food_lag8,crb_food_lag9,crb_food_lag10,crb_metal_lag1,crb_metal_lag2,crb_metal_lag3,crb_metal_lag4,crb_metal_lag5,crb_metal_lag6,crb_metal_lag7,crb_metal_lag8,crb_metal_lag9,crb_metal_lag10
month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004-01,0.76,0.68,-0.548387,-11559013.0,-22319905.0,-18028683.0,-17053471.0,-384.95,-0.0735,-0.363,-7.19247,-6.29474,-1.25943,-1.15,-1.4,0.0,-1585.77401,198.698508,-1501.06827,-200.182185,-83.222143,34.31,31.28,-2274.0,65.64871,1.56545,3.18406,-7.88372,150001,6245,212.0,2481.49,0.4,49.609174,-2.34359,8.9,14.696341,348.7366,7.70,18.70,...,11742.2420,1465.1597,-1598.9308,-1981.6094,-1224.5714,-25.6229,-1652.8386,-1673.4699,-3627.1272,3180.858937,7.29,-3.11,11.29,13.93,3.04,2.08,-1.42,2.91,-0.49,-2.07,-6.41,2.29,3.54,13.32,5.53,-2.61,-3.46,3.01,0.85,0.72,27.42,2.99,17.20,15.18,-2.05,9.59,-0.18,2.52,1.59,-3.76
2004-02,0.61,0.90,0.000000,1650795.0,11117714.0,1275340.0,-2131936.0,-96.42,0.0785,1.699,-0.78942,-1.02353,-3.21265,0.86,0.8,0.0,-470.18700,-351.442000,132.56500,-97.562000,-153.748000,34.68,30.86,-6479.7,65.72891,-2.12006,-8.67680,-5.52757,148766,7181,370.0,29.00,0.3,121.394093,1.00000,-0.8,15.073171,0.0000,10.11,14.74,...,348.7366,11742.2420,1465.1597,-1598.9308,-1981.6094,-1224.5714,-25.6229,-1652.8386,-1673.4699,-3627.127200,7.70,7.29,-3.11,11.29,13.93,3.04,2.08,-1.42,2.91,-0.49,18.70,-6.41,2.29,3.54,13.32,5.53,-2.61,-3.46,3.01,0.85,15.10,27.42,2.99,17.20,15.18,-2.05,9.59,-0.18,2.52,1.59
2004-03,0.47,0.40,-0.112903,-1575433.0,-1282702.0,300227.0,-4199993.0,387.24,-0.0248,-2.777,1.53498,2.86590,15.02434,12.53,-0.7,0.0,776.15800,171.100000,466.99600,56.588000,81.474000,36.74,33.63,10259.0,82.95766,13.99826,25.26343,13.95086,185509,9212,402.0,2375.73,0.2,-166.416169,-5.00000,-10.1,16.078049,-10382.5684,4.04,2.71,...,0.0000,348.7366,11742.2420,1465.1597,-1598.9308,-1981.6094,-1224.5714,-25.6229,-1652.8386,-1673.469900,10.11,7.70,7.29,-3.11,11.29,13.93,3.04,2.08,-1.42,2.91,14.74,18.70,-6.41,2.29,3.54,13.32,5.53,-2.61,-3.46,3.01,24.27,15.10,27.42,2.99,17.20,15.18,-2.05,9.59,-0.18,2.52
2004-04,0.37,0.21,-0.270430,539382.0,-378329.0,-4736024.0,-2778970.0,-2535.03,0.0005,-1.562,0.13157,-0.81883,-5.50945,-4.62,-0.9,0.0,761.85300,208.669000,278.87800,243.212000,31.095000,36.75,33.59,-2418.6,76.27416,-4.78950,-8.60280,-5.81031,163375,8611,-33.0,-1911.68,0.2,21.276658,-4.00000,-5.3,16.454878,-339.4192,-0.45,6.35,...,-10382.5684,0.0000,348.7366,11742.2420,1465.1597,-1598.9308,-1981.6094,-1224.5714,-25.6229,-1652.838600,4.04,10.11,7.70,7.29,-3.11,11.29,13.93,3.04,2.08,-1.42,2.71,14.74,18.70,-6.41,2.29,3.54,13.32,5.53,-2.61,-3.46,10.95,24.27,15.10,27.42,2.99,17.20,15.18,-2.05,9.59,-0.18
2004-05,0.51,0.54,-0.116667,1252881.0,13051576.0,26890443.0,11143701.0,-62.56,0.1944,4.213,1.09641,1.33059,4.81835,-1.33,0.8,20.0,-801.67600,-331.666000,-54.57900,-341.355000,-74.077000,40.28,37.57,1947.6,81.42104,5.58800,1.67710,3.80982,172380,9116,260.0,1488.90,0.2,14.892162,17.00000,16.2,15.324390,238.2590,-5.31,-15.02,...,-339.4192,-10382.5684,0.0000,348.7366,11742.2420,1465.1597,-1598.9308,-1981.6094,-1224.5714,-25.622900,-0.45,4.04,10.11,7.70,7.29,-3.11,1

In [16]:
# Cria diferentes DataFrames, cada um com uma quantidade diferente de lags
# Variam de 0 a 10 lags
dfs = []
for i in range(1, 12):
    new_df = df.copy()
    for col in df:
        if "lag" in col:
            _, lag_n = col.split("lag")
            if int(lag_n) >= i:
                new_df = new_df.drop(columns=col)
    dfs.append(new_df)
new_df.head()

,ipca,ipca_15,selic,m1,m2,m3,m4,ibovespa,cambio,cambio_real,ind_varejo,ind_varejo_ampl,ind_industria,ibc_br,capacidade_industria,salario_minimo,energia_total,energia_residencial,energia_industrial,energia_comercial,energia_outros,oil_usa,oil_eu,pib,bens_capital,bens_int,bens_cons_d,bens_cons_nd,prod_veiculos,prod_caminhoes,prod_onibus,result_primario,el_nino,epu_br,epu_usa,ind_conf_cons,desemprego,rendimento,crb_cmdt,crb_food,...,rendimento_lag1,rendimento_lag2,rendimento_lag3,rendimento_lag4,rendimento_lag5,rendimento_lag6,rendimento_lag7,rendimento_lag8,rendimento_lag9,rendimento_lag10,crb_cmdt_lag1,crb_cmdt_lag2,crb_cmdt_lag3,crb_cmdt_lag4,crb_cmdt_lag5,crb_cmdt_lag6,crb_cmdt_lag7,crb_cmdt_lag8,crb_cmdt_lag9,crb_cmdt_lag10,crb_food_lag1,crb_food_lag2,crb_food_lag3,crb_food_lag4,crb_food_lag5,crb_food_lag6,crb_food_lag7,crb_food_lag8,crb_food_lag9,crb_food_lag10,crb_metal_lag1,crb_metal_lag2,crb_metal_lag3,crb_metal_lag4,crb_metal_lag5,crb_metal_lag6,crb_metal_lag7,crb_metal_lag8,crb_metal_lag9,crb_metal_lag10
month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004-01,0.76,0.68,-0.548387,-11559013.0,-22319905.0,-18028683.0,-17053471.0,-384.95,-0.0735,-0.363,-7.19247,-6.29474,-1.25943,-1.15,-1.4,0.0,-1585.77401,198.698508,-1501.06827,-200.182185,-83.222143,34.31,31.28,-2274.0,65.64871,1.56545,3.18406,-7.88372,150001,6245,212.0,2481.49,0.4,49.609174,-2.34359,8.9,14.696341,348.7366,7.70,18.70,...,11742.2420,1465.1597,-1598.9308,-1981.6094,-1224.5714,-25.6229,-1652.8386,-1673.4699,-3627.1272,3180.858937,7.29,-3.11,11.29,13.93,3.04,2.08,-1.42,2.91,-0.49,-2.07,-6.41,2.29,3.54,13.32,5.53,-2.61,-3.46,3.01,0.85,0.72,27.42,2.99,17.20,15.18,-2.05,9.59,-0.18,2.52,1.59,-3.76
2004-02,0.61,0.90,0.000000,1650795.0,11117714.0,1275340.0,-2131936.0,-96.42,0.0785,1.699,-0.78942,-1.02353,-3.21265,0.86,0.8,0.0,-470.18700,-351.442000,132.56500,-97.562000,-153.748000,34.68,30.86,-6479.7,65.72891,-2.12006,-8.67680,-5.52757,148766,7181,370.0,29.00,0.3,121.394093,1.00000,-0.8,15.073171,0.0000,10.11,14.74,...,348.7366,11742.2420,1465.1597,-1598.9308,-1981.6094,-1224.5714,-25.6229,-1652.8386,-1673.4699,-3627.127200,7.70,7.29,-3.11,11.29,13.93,3.04,2.08,-1.42,2.91,-0.49,18.70,-6.41,2.29,3.54,13.32,5.53,-2.61,-3.46,3.01,0.85,15.10,27.42,2.99,17.20,15.18,-2.05,9.59,-0.18,2.52,1.59
2004-03,0.47,0.40,-0.112903,-1575433.0,-1282702.0,300227.0,-4199993.0,387.24,-0.0248,-2.777,1.53498,2.86590,15.02434,12.53,-0.7,0.0,776.15800,171.100000,466.99600,56.588000,81.474000,36.74,33.63,10259.0,82.95766,13.99826,25.26343,13.95086,185509,9212,402.0,2375.73,0.2,-166.416169,-5.00000,-10.1,16.078049,-10382.5684,4.04,2.71,...,0.0000,348.7366,11742.2420,1465.1597,-1598.9308,-1981.6094,-1224.5714,-25.6229,-1652.8386,-1673.469900,10.11,7.70,7.29,-3.11,11.29,13.93,3.04,2.08,-1.42,2.91,14.74,18.70,-6.41,2.29,3.54,13.32,5.53,-2.61,-3.46,3.01,24.27,15.10,27.42,2.99,17.20,15.18,-2.05,9.59,-0.18,2.52
2004-04,0.37,0.21,-0.270430,539382.0,-378329.0,-4736024.0,-2778970.0,-2535.03,0.0005,-1.562,0.13157,-0.81883,-5.50945,-4.62,-0.9,0.0,761.85300,208.669000,278.87800,243.212000,31.095000,36.75,33.59,-2418.6,76.27416,-4.78950,-8.60280,-5.81031,163375,8611,-33.0,-1911.68,0.2,21.276658,-4.00000,-5.3,16.454878,-339.4192,-0.45,6.35,...,-10382.5684,0.0000,348.7366,11742.2420,1465.1597,-1598.9308,-1981.6094,-1224.5714,-25.6229,-1652.838600,4.04,10.11,7.70,7.29,-3.11,11.29,13.93,3.04,2.08,-1.42,2.71,14.74,18.70,-6.41,2.29,3.54,13.32,5.53,-2.61,-3.46,10.95,24.27,15.10,27.42,2.99,17.20,15.18,-2.05,9.59,-0.18
2004-05,0.51,0.54,-0.116667,1252881.0,13051576.0,26890443.0,11143701.0,-62.56,0.1944,4.213,1.09641,1.33059,4.81835,-1.33,0.8,20.0,-801.67600,-331.666000,-54.57900,-341.355000,-74.077000,40.28,37.57,1947.6,81.42104,5.58800,1.67710,3.80982,172380,9116,260.0,1488.90,0.2,14.892162,17.00000,16.2,15.324390,238.2590,-5.31,-15.02,...,-339.4192,-10382.5684,0.0000,348.7366,11742.2420,1465.1597,-1598.9308,-1981.6094,-1224.5714,-25.622900,-0.45,4.04,10.11,7.70,7.29,-3.11,1

In [17]:
for d in dfs:
    print(len(d.columns))

41
82
123
164
205
246
287
328
369
410
451


In [30]:
results_dict = {}
y_preds = {}
for i, d in enumerate(dfs):
    regressor = RandomForestRegressor(max_features="sqrt")
    forecaster = make_reduction(regressor)
    y_train, y_test, y_pred = utils.evaluate_pipeline(forecaster, d)

    y_preds[f'lags_{i}'] = y_pred
    y_pred.name = "ipca"
    
    mae = MeanAbsoluteError()
    mse = MeanSquaredError()
    mape = MeanAbsolutePercentageError()

    results_dict[f'lags_{i}'] = [
        mae(y_test, y_pred),
        mse(y_test, y_pred),
        mape(y_test, y_pred)
    ]

e:\tcc\utils.py:81: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_pred = pd.Series([value[0] for _, value in df['y_pred'].items()])
e:\tcc\utils.py:83: FutureWarning: The 'freq' keyword in the PeriodArray constructor is deprecated and will be removed in a future version. Pass 'dtype' instead
  period_array = pd.arrays.PeriodArray(period_list, freq="M")
e:\tcc\utils.py:81: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_pred = pd.Series([value[0] for _, value in df['y_pred'].items()])
e:\tcc\utils.py:83: FutureWarning: The 'freq' keyword in the PeriodArray constructor is deprecated and will be removed in a future ve

In [29]:
res_df = pd.DataFrame(results_dict, index=['MAE', 'MSE', 'MAPE'])
res_df

,lags_0,lags_1,lags_2,lags_3,lags_4,lags_5,lags_6,lags_7,lags_8,lags_9,lags_10
MAE,0.297677,0.293500,0.307882,0.301411,0.311671,0.298461,0.300819,0.301268,0.306182,0.305328,0.306446
MSE,0.156921,0.157981,0.163154,0.161128,0.168675,0.158497,0.159395,0.159316,0.164732,0.161392,0.164384
MAPE,1.460625,1.494441,1.477703,1.643038,1.581758,1.453864,1.431539,1.453532,1.523021,1.440831,1.549588
